In [ ]:
!pip install datamodel_code_generator
!pip install httpcore
!pip install httpcore==0.15.0 httpx pymongo googletrans
!pip install openai
!pip install func_timeout

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 896.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.9 MB/s eta 0:00:00
  Attempting uninstall: inflect
    Found existing installation: inflect 7.4.0
    Uninstalling inflect-7.4.0:
      Successfully uninstalled inflect-7.4.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of httpx to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.p

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = 'cuda'

In [ ]:


model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(device)

model_outputs = model.generate(
    model_inputs,
    max_new_tokens=1024,
    top_p=0.7,
    temperature=0.7
)

output_token_ids = [
    model_outputs[i][len(model_inputs[i]):] for i in range(len(model_inputs))
]

responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
print(responses)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


思考：回复文本中没有明显的数学计算或代码，因此不需要使用calculator或code_interpreter工具。回复文本是一个完整的句子，可以直接进行核查。使用match工具来检查回复是否与历史事实相符。

行为：match(sentence="印度尼西亚在独立之前被荷兰殖民统治了约350年。", context="")


In [ ]:
import json
import torch
from tools import *
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

# Initialize model and tokenizer
def init_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map='auto', trust_remote_code=True)
    model.generation_config = GenerationConfig.from_pretrained(model_path)

    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    return model, tokenizer

# Dump the data to a JSON lines file
def dump_jsonl(data, output_path, append=False):
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        json_record = json.dumps(data, ensure_ascii=False)
        f.write(json_record + '\n')

# Main function for generating responses
def generate_single_question_answer(model_path, question, answer, output_file):
    model, tokenizer = init_model(model_path)

    temp = ("你是一个通过特定的框架检测回复文本中的幻象的智能体。下面是检测框架的详细说明。\n"
            "首先，你需要判断是否要将输入中的回复文本拆分为句子列表。 你可以使用拆分句子的工具。"
            "如果需要拆分，需要对每个句子逐一进行核查；否则就对整个回复文本进行核查。"
            "你可以选择适当的事实核查工具来获取用于核查的相关信息和知识然后使用匹配工具输出判断结果或者直接输出判断结果。"
            "如果不使用match工具而直接输出判断结果，则需要在思考中输出label。"
            "存在错误输出\"label = 1\"；不存在错误输出\"label = 0\"。"
            "核查完毕后，你需要在思考中反思所有检测结果并输出label，在行为中调用get_answer()输出最终的检测结果，如果存在幻象一并输出幻象内容和证据。\n\n"
            "分句工具：\nsplit_text(text: str) → sentence_list\n"
            "输入是文本，该函数将文本分割成句子列表。\n\n"
            "事实核查工具：\nweb_search(sentence: str) → fact\n"
            "输入是一个句子，该函数使用搜索引擎来搜索相关信息。调用web_search后必须接着调用match工具来判断言判断的回复与检索到的信息是否匹配。\n\n"
            "calculator(sentence: str, formula: str) → result, label\n"
            "输入是需要检查的公式，此函数使用计算器来获取计算结果并判断得到的结果是否与句子匹配。"
            "如果匹配label为0，否则为1。有效的运算符有 +、-、*、/ 和 (, )。"
            "例如，合法的输入可以是“(1 + 2) * 3”。如果输入为方程，需要将其转换为不含未知数的算式。\n\n"
            "word_count(length: int, text: str) → count, label\n"
            "输入文本的指定字数和一段文本。该函数计算这段文本的字数并输出为count。如果字数不符合要求，输出label为1，否则为0。\n\n"
            "code_interpreter() → label\n"
            "该函数检查代码中是否能够正确执行。如果能正确执行，输出标签为0，否则为1。\n\n"
            "匹配工具：\nmatch(sentence: str, context:str) → label\n"
            "输入是一个句子以及相应的上下文。上下文可以是问题和回复中的检测句子之前的内容。"
            "该函数检查句子中是否存在答非所问或自相矛盾的情况。如果有，则输出标签为1，否则为0。"
            "如果你认为match的输出是错误的，可以在思考中修正label，例如如果你认为match输出的\"label = 0\"是错误的，可以在思考中输出\"label = 1\"。\n\n"
            "每次轮到你回复时，你必须严格遵循以下格式给出你的思考和行为："
            "\"思考：你的思考过程。\n行为：工具调用。如match(sentence=\"...\", context=\"...\")\"，"
            "其中思考部分是你的规划内容，行为部分必须为一个工具调用指令。"
            "每次你调用工具后，我会以这种格式为你提供结果：“观察：工具的输出结果”。")

    messages = []
    query = {"role": "user", "content": temp + "问题：" + question + " 回复：" + answer}
    messages.append(query)

    # Prepare the input for the model
    model_inputs = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(model.device)

    # Generate model outputs
    model_outputs = model.generate(
        model_inputs['input_ids'],
        max_new_tokens=1024,
        top_p=0.7,
        temperature=0.7
    )

    # Process outputs
    output_token_ids = model_outputs[:, len(model_inputs['input_ids'][0]):]
    responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
    print(responses)

    # Rest of the logic based on responses
    if "行为：" in responses:
        action = responses.split("行为：")[-1].strip(".").strip()
        if "\n" in action:
            action = action.split("\n")[0]
        responses = responses.split("行为：")[0] + "行为：" + action
    messages.append({"role": "assistant", "content": responses})

    label_list = []
    fact_list = []
    sen_list = []
    web_fact = ""

    try:
        fnum = 0
        while True:
            response = ""
            response0 = "观察："
            action = responses.split("行为：")[-1].strip(".").strip(":").strip()
            print(action)
            if "split" in action:
                sentences = eval(action)
                response = {"role": "user", "content": response0 + str(sentences)}
            elif "calculator" in action:
                result, label = eval(action)
                response = {"role": "user", "content": response0 + result + ", label = " + str(label)}
                sentence = action.split("sentence=")[-1].split("\"")[1]
                sen_list.append(sentence)
                fact_list.append(result)
                label_list.append(label)
            elif "web_search" in action:
                fact = eval(action)
                response = {"role": "user", "content": response0 + fact}
                web_fact = fact
            elif "match" in action:
                label = eval(action)
                text = "label = " + str(label)
                response = {"role": "user", "content": response0 + text}
                sentence = action.split("sentence=")[-1].split("\"")[1]
                sen_list.append(sentence)
                fact = action.split("context=")[-1].strip("\")")
                fact_list.append(fact)
                label_list.append(label)
            elif "code_interpreter" in action:
                label, report = eval(action)
                text = "label = " + str(label)
                response = {"role": "user", "content": response0 + text}
                sen_list.append(answer)
                fact = report
                fact_list.append(fact)
                label_list.append(label)
            elif "word_count" in action:
                text = answer
                count, label = eval(action)
                response = {"role": "user", "content": response0 + "这段文本的字数是" + str(count) + "，label=" + str(label)}
                sen_list.append(answer)
                label_list.append(label)
                fact_list.append("这段文本的字数是" + str(count))
            elif "get_answer" in action:
                fnum += 1
                if fnum > 5:
                    break
                if not sen_list:
                    sen_list.append(answer)
                    if web_fact:
                        fact_list.append(web_fact)
                if not label_list:
                    response = {"role": "user", "content": "观察：未检测到label，请输出label并重新调用get_answer()。"}
                    print(response)
                    messages.append(response)
                    # Prepare the input for the model again


                    # Generate model outputs again
                    model_outputs = model.generate(
                        messages,
                        max_new_tokens=1024,
                        top_p=0.7,
                        temperature=0.7
                    )

                    # Process outputs again
                    output_token_ids = model_outputs[:, len(model_inputs['input_ids'][0]):]
                    responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]

                    if "行为：" in responses:
                        action = responses.split("行为：")[-1].strip(".").strip()
                        thought = responses.split("行为：")[0].strip(".").strip()
                        if "label=1" in thought:
                            if label_list:
                                label_list[-1] = 1
                            else:
                                label_list.append(1)
                        elif "label=0" in thought:
                            if label_list:
                                label_list[-1] = 0
                            else:
                                label_list.append(0)
                        if "\n" in action:
                            action = action.split("\n")[0]
                        responses = responses.split("行为：")[0] + "行为：" + action
                    messages.append({"role": "assistant", "content": responses})
                    continue

                final_answer = get_answer(sen_list, label_list, fact_list)
                messages.append({"role": "user", "content": "观察：" + final_answer})
                break
            else:
                print("No such tool, failed!")
                break

            messages.append(response)
            # Prepare the input for the model again


            # Generate model outputs again
            model_outputs = model.generate(
                messages
            )

            # Process outputs again
            output_token_ids = model_outputs[:, len(model_inputs['input_ids'][0]):]
            responses = tokenizer.batch_decode(output_token_ids, skip_special_tokens=True)[0]
            if "行为：" in responses:
                action = responses.split("行为：")[-1].strip(".").strip()
                thought = responses.split("行为：")[0].strip(".").strip()
                if "label=1" in thought:
                    label_list[-1] = 1
                elif "label=0" in thought:
                    label_list[-1] = 0
                if "\n" in action:
                    action = action.split("\n")[0]
                responses = responses.split("行为：")[0] + "行为：" + action
            messages.append({"role": "assistant", "content": responses})
            print(responses)
    except Exception as e:
        print(e)
        d = {"final_answer": "Error", "trajectory": messages}
        dump_jsonl(d, f"{output_file}l", append=True)
        return

    final_answer = final_answer.split("观察：")[-1].strip()
    d = {"final_answer": final_answer, "trajectory": messages}
    dump_jsonl(d, output_file, append=True)

# In Jupyter, you can run the following cell to pass arguments and run the script
# Set your model path and output file path
model_path = "openbmb/MiniCPM3-4B"
output_file = "output.json"


In [ ]:
# Example question-answer pair
question = "Is 7 a prime number?"
answer = "The number 7 is not a prime number. This is because it is divisible by 7 and 7/7 = 1."

# Run the generation process for a single question-answer pair
generate_single_question_answer(model_path, question, answer, output_file)

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1462: UserWarning: Current model requires 79692384 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [ ]:
|